In [1]:
import os
import json
from opensearchpy import OpenSearch, RequestsHttpConnection, helpers
from opensearchpy.exceptions import TransportError, ConnectionError as OSCxnError

import boto3
from requests_aws4auth import AWS4Auth
from openai import OpenAI, OpenAIError
from config import *
import time
import logging

from email import message_from_string
from email import policy
from email.utils import make_msgid
from src.services.data_extraction import fast_stream_first_n
from src.services.querying import *


In [2]:
retrieved_ids = []
memory = None

In [3]:
# hits, ids = knn_search(query, retrieved_ids)

# print("Newly Retrieved IDs:")
# for i in hits:
#     print(f"Subject: {i["_source"]["subject"]}")
#     print(f"Body: {i["_source"]["summary_text"][:50]}")
#     print()

# print("\nExcluded IDs:")
# for i in retrieved_ids:
#     print(i)

# retrieved_ids.extend(ids)

In [14]:
query = "Tell me about both of them. What do they do?"


prompt, response, memory, retrieved_ids = answer_query(query, memory, retrieved_ids)

In [16]:
memory = memory
retrieved_ids.extend(retrieved_ids)

print(response)

Based on the information available to me, here is a summary of the roles and responsibilities of Sarah Hawkins and Abdul Sissi:

**Sarah Hawkins:**
Sarah Hawkins is the Import & Export Clerk at Redcoat Express. Her role involves coordinating shipment bookings and ensuring compliance with regulations, particularly those concerning hazardous and restricted articles. She is responsible for communicating with external partners such as shipping companies and airlines to facilitate logistics operations. Her tasks include addressing rate inquiries, managing booking details, and proactively resolving potential issues related to allotments that might affect the booking process. Her attention to detail and proactive approach are crucial for efficient cargo handling at Redcoat Express (Abdul Sissi and Sarah Hawkins, "Booking Details for this Week," May 2025; Sarah Hawkins and Lufthansa Cargo, "Rate LHR - FNA 020-65655391," April 2025).

**Abdul Sissi:**
Abdul Sissi is associated with Alie Baba Sh